In [ ]:
import pandas as pd
import numpy as np
import cv2
from Common import Common
import seaborn as sns
import matplotlib.pyplot as plt
from patsy import dmatrices
import statsmodels.api as sm

df_original = pd.read_csv('spreadsheets/BUSL_Notes.csv', skiprows=1)
df_original = df_original.dropna(axis="rows", how="all").dropna(axis="columns", how="all")
df_radiomics = pd.read_csv('spreadsheets/Radiomics.csv')

Drop some unhelpful columns and reduce the dataset to only the numeric values

In [ ]:
df = df_radiomics.drop(columns = ['original_firstorder_Minimum','original_firstorder_Maximum','original_firstorder_Range'])
df_numeric = df._get_numeric_data() #drop non-numeric cols
df_numeric = df_numeric.drop(columns=['filename'])
df_numeric.head()

Prepare to drop any columns that have a correlation above a certain threshold (0.8 in this case)

In [ ]:
def identify_correlated(df, threshold):
    """
    A function to identify highly correlated features.
    """
    # Compute correlation matrix with absolute values
    matrix = df.corr().abs()
    
    # Create a boolean mask
    mask = np.triu(np.ones_like(matrix, dtype=bool))
    
    # Subset the matrix
    reduced_matrix = matrix.mask(mask)
    
    # Find cols that meet the threshold
    to_drop = [c for c in reduced_matrix.columns if \
              any(reduced_matrix[c] > threshold)]
    
    return to_drop

In [ ]:
to_drop = identify_correlated(df_numeric, threshold=.8)
df_reduced = df_numeric.drop(to_drop, axis=1)

Join the two data frames

In [ ]:
def expand_filename (row):
  fn = row["filename"]
  return "PA" + str(fn)

In [ ]:
df_reduced["SampleName"] = df.apply(lambda row: expand_filename(row), axis=1)

In [ ]:
# df.join(other.set_index('key'), on='key')
df_joined = df_reduced.join(df_original.set_index('Sample name'), lsuffix='_rad', rsuffix='_rle', on='SampleName')

In [ ]:
df_joined.head()

In [ ]:
df_joined.to_csv('spreadsheets/joined.csv', index=False)

Add the actual histology and filename back in

In [ ]:
df_reduced["histology_actual"] = df["Histology_Actual"]
df_reduced["histology_rle"] = df["Histology_RLE"]
df_reduced["filename"] = df["filename"]
df_reduced["filename"] = df["filename"]
df_reduced["filename"] = df["filename"]
df_reduced.insert(0, 'histology_rle', df_reduced.pop('histology_rle'))
df_reduced.insert(0, 'histology_actual', df_reduced.pop('histology_actual'))
df_reduced.insert(0, 'filename', df_reduced.pop('filename'))

Add the shape back in

In [ ]:
df_reduced["ShapeACR"] = df_joined["ShapeACR "]
df_reduced["ShapeTS"] = df_joined["ShapeTS"]

Here are the functions for quantifying the marginal zone abruptness

In [ ]:
# White points with at least one black neighbor
def has_one_or_two_black_neighbors(gt, y, x):
  arr = [gt[y, x-1] == 0, gt[y, x+1] == 0, gt[y-1, x] == 0,gt[ y+1, x] == 0]
  count = arr.count(True)
  return count == 1 or count == 2

def get_marginal_zone_acr(bus, gt, kernel_size):
  i_max = 10
  prev_mean = 255
  threshold = 0.95
  show_images = False
  img = np.uint8(gt)
  for i in range(i_max):
    kernel = np.ones((kernel_size,kernel_size),np.uint8)
    erosion = cv2.erode(img,kernel,iterations = i)
    mask = bus.copy()
    # mask = (marginal % 254) * bus
    mask = (erosion % 254) * bus
    this_mean = mask[np.nonzero(mask)].mean()
    if this_mean/prev_mean > threshold or this_mean < 15:
      #print("stopping at " + str(i))
      return i
      break
    prev_mean = this_mean
    if show_images:
      plt.figure(figsize=(8,12))
      plt.title(str(this_mean))
      plt.imshow(mask, cmap=plt.cm.gray, vmax=255)
  return i_max


def get_abrupt_boundary_metric(bus, gt):
  gy_bus, gx_bus = np.gradient(bus)

  out_window = 3
  in_window = 7
  abrupt_threshold = 20
  abrupt_pixels = 0
  max_gradients = []
  white_points = np.column_stack(np.where(gt == 255))
  border_points = [point for point in white_points if has_one_or_two_black_neighbors(gt, point[0], point[1])]
  for point in border_points:
    # Get the x,y coords
    y, x = point[0], point[1]
    # Get the proper gradient range and look for an
    v = 0
    h = 0
    if gt[y-1, x] == 0: # Look down for the gradient
      v = 1
    if gt[y+1, x] == 0:  # Look up for the gradient
      v = -1
    if gt[y, x-1] == 0:  # Look right for the gradient
      h = 1
    if gt[y, x+1] == 0:  # Look left for the gradient
      h = -1

    v_grad = np.empty(1)
    h_grad = np.empty(1)
    if v == 1:
      v_grad = abs(gy_bus[y-out_window:y+in_window,x])
    elif v == -1:
      v_grad = abs(gy_bus[y-in_window:y+out_window,x])
    if h == 1:
      h_grad = abs(gx_bus[y, x-out_window:x+in_window])
    elif h == -1:
      h_grad = abs(gx_bus[y, x-in_window:x+out_window])

    if (len(v_grad) > 0 and max(v_grad) >= abrupt_threshold) or (len(h_grad) > 0 and max(h_grad) >= abrupt_threshold):
      abrupt_pixels += 1

    max_gradients.append(max(abs(v_grad+h_grad)))

  return abrupt_pixels/len(border_points), np.average(max_gradients)

...and some code for quantifying orientation

In [ ]:
def get_orientation(gt):

  # threshold
  thresh = cv2.threshold(gt, 100 , 255, cv2.THRESH_BINARY)[1]

  # find largest contour
  contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contours = contours[0] if len(contours) == 2 else contours[1]
  big_contour = max(contours, key=cv2.contourArea)

  # fit contour to ellipse and get ellipse center, minor and major diameters and angle in degree 
  # the angle here refers to the angle with the middle of the x-axis, so it could run 
  # from 0 (straight to the left) to 180 degrees (straight to the right), with 90 degrees being perpendicular to the x-axis
  ellipse = cv2.fitEllipse(big_contour)
  (xc,yc),(d1,d2),angle = ellipse

  # We are really oly interested in the angle's absolute deviation from vertical.
  angle = abs(90-angle)
  return angle

Add columns to the data frame for abruptness and orientation

In [ ]:
root_dir = '/content/gdrive/MyDrive/BUS_Project_Home/Share_with_group/Adam_Silberfein'
import sys
sys.path.append(root_dir)
import os

for i, row in df_reduced.iterrows():
  filename = str(row["filename"]).zfill(3)

  try:
    bus, gt, og = Common.get_images(row["filename"])
    marginal_zone = get_marginal_zone_acr(bus, gt, 5)
    abruptness, avg_max_gradient = get_abrupt_boundary_metric(bus, gt)
    orientation = get_orientation(gt)
    df_reduced.at[i, "abruptness"] = abruptness
    df_reduced.at[i, "avg_max_gradient"] = avg_max_gradient
    df_reduced.at[i, "orientation"] = orientation
  except:
    print("Error on ", filename)
    continue
  
  

In [ ]:
df_reduced = df
for i, row in df_reduced.iterrows():
  filename = str(row["filename"]).zfill(3)
  print(filename)
  bus, gt, og = Common.get_images(filename)
  orientation = get_orientation(gt)
  df_reduced.at[i, "orientation"] = orientation


In [ ]:
df_reduced